In [1]:
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [10]:
#WHERE WE EXTRACT TABLE NAME FROM CSV
from bs4 import BeautifulSoup
from tika import parser
import os
import re
import camelot
from fuzzywuzzy import fuzz
from datetime import datetime

C:\Users\yazdsous\AppData\Local\Continuum\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
#######################FUNCTIONS##################################################
def get_table_titles(page:int, df:pd.DataFrame) -> list():#pd.DataFrame:
    tbl_names = list()
    tbl_names = list(df[df['page_number']== page]['final_table_title'])
                
    return tbl_names
#********************************************************************************#
def replace_chars(text:str) -> str:
    chars_0 = ['\n',':']
    chars_1 = ['/','\\']
    for c in chars_0:
        text = text.replace(c, ' ')
    for cc in chars_1:
        text = text.replace(cc,'_')
    return text
#********************************************************************************#
def replace_chars_strings(lst:list) -> list:
    new_lst = []
    for itm in lst:
        new_lst.append(replace_chars(itm))       
    return new_lst



In [16]:
#Main Function
def extract_tables(file:str(),df:pd.DataFrame):    
    start_time = datetime.now()

    file_path = 'F:/Environmental Baseline Data/Version 4 - Final/PDF/{}'.format(file)

    file_name = file_path.split('/')[-1].replace('.pdf','')

    data = parser.from_file(file_path,xmlContent=True)
    #raw_xml = parser.from_file('A6T2V6.pdf', xmlContent=True)

    #xml tag <div> splitting point for pages
    soup = BeautifulSoup(data['content'], 'lxml')
    pages = soup.find_all('div', attrs={'class': 'page'})

    title_dict = dict()
    start_time = datetime.now()
    for ind, page in enumerate (pages):
        pg_num = ind
        chars = []
        #camelot table objects for each page of the pdf
        try:
            tables = camelot.read_pdf(file_path, pages = str(pg_num+1), flag_size=True, copy_text=['v'],strip_text = '\n',line_scale=40, f = 'csv',flavour = 'stream')  #loop len(tables)
        except:
            continue
        #get table names in page == pg_num by parsing get_table_titles() function
        title_lst_raw = get_table_titles(pg_num+1,df)
        title_lst = replace_chars_strings(title_lst_raw)
        #get total number of table objects detected by Camelot in page == pg_num
        tb_num = tables.n

        #VIEW
        print(title_lst)  
        #if Camelot returns NO table on the page continue the loop and go to the next page
        if tb_num == 0:
            print("No table on page "+ str(pg_num+1) + " is detected")
            continue
        #if whitespace of the detected table is larger than 69% of the entire table and there is only
        #one table on that page, identify this as figure and continute the loop and go to the next page
        elif tb_num == 1 and (tables[0].parsing_report)['whitespace'] > 69.0:
            print("Page {} contains an image".format((tables[0].parsing_report)['page'] ))  
            continue
        #in case only one table is present on the page,
        elif tb_num == 1:
            #this block distills the dataframe with proper column names
            df_tb = tables[0].df
            df_tb = df_tb.replace('/na', '_', regex = True)
            colname = df_tb.iloc[0].str.replace('\n',' ',regex=True)
            df_tb.columns = colname
            df_tb = df_tb[1:]   
            #df_tb = df_tb.iloc[1:]
            #in case no title is extracted from this page but we know that there is one table
            if len(title_lst) == 0:
                #let's say we are on page number x and this if statement checks whether page number x-1 contianed a table or not
                #if the result is TRUE we assign the title of last table on previous page to this page's title-less table
                if (pg_num-1 in title_dict):
                    #find the list of tables on the previous page
                    lst_tbl = (title_dict.get(pg_num-1))[-1]
                    lst_tbl_df = lst_tbl[1]
                    #find similarity score of column names of table on page x and page x-1
                    col_concat_curr = list(df_tb.columns.values)
                    ccc = ''.join(col_concat_curr)
                    col_join_curr = (ccc.replace(' ','')).replace('\n','')
                    col_concat_prev = list(lst_tbl_df.columns.values)
                    ccp = ''.join(col_concat_prev)
                    col_join_prev = (ccp.replace(' ','')).replace('\n','')  
                    ratio_similarity = fuzz.token_sort_ratio(col_join_curr, col_join_prev)
                    #check if the columns of the last table on the previous page are the same as the table on this page
                    if (len((set(lst_tbl_df.columns)).difference(set(df_tb.columns))) == 0) or (len(set(lst_tbl_df.columns))== len(set(df_tb.columns))) or (ratio_similarity > 89):
                        xl_name = lst_tbl[2]
                        chars.append([0,df_tb,xl_name])
                        xlsx_name = file_name + '_' + xl_name +'_'+str(pg_num+1)+'_'+str(1)+ '.csv'
                        df_tb.to_csv(xlsx_name, index = False, encoding='utf-8-sig')     
                    else:
                        xl_name = file_name
                        chars.append([0,df_tb,xl_name])
                        xlsx_name = file_name + '_' +str(pg_num+1)+'_'+str(1)+ '.csv'
                        df_tb.to_csv(xlsx_name, index = False, encoding='utf-8-sig')
                    title_dict[pg_num] = chars

                else:
                    xl_name = file_name       
                    chars.append([0,df_tb,xl_name])
                    xlsx_name = file_name + '_' +str(pg_num+1)+'_'+str(1)+ '.csv'
                    df_tb.to_csv(xlsx_name, index = False, encoding='utf-8-sig')
                    title_dict[pg_num] = chars
            else:
                xl_name = title_lst[0]
    #                xl_name = xl_name.replace('/','_')
    #                xl_name = xl_name.replace(':','')
                #store page number, index of the table, and its name in a dictionary
                chars.append([0,df_tb,xl_name])
                xlsx_name = file_name + '_' + xl_name +'_'+str(pg_num+1)+'_'+str(1)+ '.csv'
                df_tb.to_csv(xlsx_name, index = False, encoding='utf-8-sig')
                title_dict[pg_num] = chars

        else:
            if len(title_lst) >= tb_num :
                for j in range(0,tb_num):
                    df_tb = tables[j].df
                    df_tb = df_tb.replace('/na', '_', regex = True)
                    df_tb.columns = df_tb.iloc[0].str.replace('\n',' ',regex=True)
                    df_tb = df_tb[1:]
                    #df_tb = df_tb.iloc[1:]
                    xl_name = title_lst[j]
    #                    xl_name = xl_name.replace('/','_')
    #                    xl_name = xl_name.replace(':','')

                    #store page number, index of the table, and its name in a dictionary
                    chars.append([j,df_tb,xl_name])

                    xlsx_name = file_name + '_' + xl_name +'_'+str(pg_num+1)+'_'+str(j+1)+ '.csv'
                    df_tb.to_csv(xlsx_name, index = False, encoding='utf-8-sig')

                title_dict[pg_num] = chars

            elif len(title_lst) < tb_num :
                try:
                    #first case: if the first table in the page is conitnuos of what was on the previous page
                    df_tb = tables[0].df
                    df_tb = df_tb.replace('/na', '_', regex = True)
                    df_tb.columns = df_tb.iloc[0].str.replace('\n',' ',regex=True)
                    df_tb = df_tb[1:]
                    #df_tb = df_tb.iloc[1:]

                    if (pg_num-1 in title_dict):
                        #find the list of tables on the previous page
                        lst_tbl = (title_dict.get(pg_num-1))[-1]
                        lst_tbl_df = lst_tbl[1]
                        #check if the columns of the last table on the previous page are the same as the table on this page

                        col_concat_curr = list(df_tb.columns.values)
                        ccc = ''.join(col_concat_curr)
                        col_join_curr = (ccc.replace(' ','')).replace('\n','')
                        col_concat_prev = list(lst_tbl_df.columns.values)
                        ccp = ''.join(col_concat_prev)
                        col_join_prev = (ccp.replace(' ','')).replace('\n','')  
                        ratio_similarity = fuzz.token_sort_ratio(ccc, ccp)

                        if len((set(lst_tbl_df.columns)).difference(set(df_tb.columns))) == 0 or len(set(lst_tbl_df.columns))== len(set(df_tb.columns)) or ratio_similarity > 89:
                            xl_name = lst_tbl[2]                           
                            chars.append([0,df_tb,xl_name])                         
                            xlsx_name = file_name + '_' + xl_name +'_'+str(pg_num+1)+'_'+str(1)+ '.csv'
                            df_tb.to_csv(xlsx_name, index = False, encoding='utf-8-sig')

                        else:
                            xl_name = file_name   
                            chars.append([0,df_tb,xl_name])
                            xlsx_name = file_name + '_' +str(pg_num+1)+'_'+str(1)+ '.csv'
                            df_tb.to_csv(xlsx_name, index = False, encoding='utf-8-sig')
                    else:
                        xl_name = file_name       
                        chars.append([0,df_tb,xl_name])
                        xlsx_name = file_name + '_' +str(pg_num+1)+'_'+str(1)+ '.csv'
                        df_tb.to_csv(xlsx_name, index = False, encoding='utf-8-sig')

                    indx = 0
                    for j in range(1,len(title_lst)+1):
                        df_tb = tables[j].df
                        df_tb = df_tb.replace('/na', '_', regex = True)
                        df_tb.columns = df_tb.iloc[0].str.replace('\n',' ',regex=True)
                        df_tb = df_tb[1:]
                        #df_tb = df_tb.iloc[1:]
                        xl_name = title_lst[indx]
                        indx = indx + 1
    #                        xl_name = xl_name.replace('/','_')
    #                        xl_name = xl_name.replace(':','')
                        #store page number, index of the table, and its name in a dictionary
                        chars.append([j,df_tb,xl_name])

                        xlsx_name = file_name + '_' + xl_name +'_'+str(pg_num+1)+'_'+str(j+1)+ '.csv'
                        #xlsx_name = z.split('\\')[-1] + '-' + str(pg[i]+1) + str(j) + '.xlsx'
                        df_tb.to_csv(xlsx_name, index = False, encoding='utf-8-sig')

                    for j in range(len(title_lst)+1,tb_num):
                        df_tb = tables[j].df
                        df_tb = df_tb.replace('/na', '_', regex = True)
                        df_tb.columns = df_tb.iloc[0].str.replace('\n',' ',regex=True)
                        df_tb = df_tb[1:]
                        #df_tb = df_tb.iloc[1:]
                        xl_name = file_name
                        #store page number, index of the table, and its name in a dictionary
                        chars.append([j,df_tb,xl_name])                        

                        xlsx_name = file_name + '_' + xl_name +'_'+str(pg_num+1)+'_'+str(j+1)+ '.csv'
                        #xlsx_name = z.split('\\')[-1] + '-' + str(pg[i]+1) + str(j) + '.xlsx'
                        df_tb.to_csv(xlsx_name, index = False, encoding='utf-8-sig')                    
                    title_dict[pg_num] = chars

                except:
                    print("Function failed on page {}".format(pg_num+1))
                    pass

    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))                        

In [2]:
path = 'F:/Environmental Baseline Data/Version 4 - Final/Support files/Table titles raw data/final_table_titles2.csv'
df = pd.read_csv(path, usecols = ['page_number','final_table_title', 'Application title short', 'DataID_pdf', \
                                  'categories', 'Category'])
df = df[df['categories'] > 0] 
df = df[df['Category'] == 'Table']
df['final_table_title'] = df['final_table_title'].str.title()
df.head()

,page_number,final_table_title,categories,DataID_pdf,Application title short,Category
2,8,Table 1 Hydrometric Data From Discharge Statio...,0.75,1059614.pdf,Application for North Montney Project,Table
3,9,Table 2 Fish Species That May Occur In The Upp...,0.75,1059614.pdf,Application for North Montney Project,Table
4,14,Table 3 Summary Of Aquatics Field Work And Abo...,0.75,1059614.pdf,Application for North Montney Project,Table
5,17,Table 4 Summary Of Watercourse Crossings Along...,0.75,1059614.pdf,Application for North Montney Project,Table
6,18,Table 4 Summary Of Watercourse Crossings Along...,0.35,1059614.pdf,Application for North Montney Project,Table


## Application for 2021 NGTL System Expansion Project

In [4]:
hearing = 'Application for 2021 NGTL System Expansion Project' 
ngtl2021 = df[df['Application title short'] == hearing].reset_index(drop = True)
ngtl2021.head()

,page_number,final_table_title,categories,DataID_pdf,Application title short,Category
0,3,Table 5.1–1: Physical And Meteorological Data ...,1.0,3578647.pdf,Application for 2021 NGTL System Expansion Pro...,Table
1,4,Table 5.1–2: Biophysical Regions In Project Area,1.0,3578647.pdf,Application for 2021 NGTL System Expansion Pro...,Table
2,6,Table 5.1–3: Uppermost Bedrock Formation,1.0,3578647.pdf,Application for 2021 NGTL System Expansion Pro...,Table
3,7,Table 5.1–4: Surficial Geology,1.0,3578647.pdf,Application for 2021 NGTL System Expansion Pro...,Table
4,10,Table 5.1–5: Stations Used To Provide Climate ...,1.0,3578647.pdf,Application for 2021 NGTL System Expansion Pro...,Table


### Change this folder to the path were you want the tables saved

In [ ]:
os.chdir(r'F:\Environmental Baseline Data\Version 4 - Final\CSV\ngtl2021')

### Change the dataframe name accordingly

In [6]:
file = ngtl2021['DataID_pdf'][0]
file

'3578647.pdf'

### call the main function -- pass filename and dataframe as function arguments

In [17]:
extract_tables(file,ngtl2021)

[]
No table on page 1 is detected
[]
No table on page 2 is detected
['Table 5.1–1  Physical And Meteorological Data Sources']
['Table 5.1–2  Biophysical Regions In Project Area']
[]
Page 5 contains an image
['Table 5.1–3  Uppermost Bedrock Formation']
['Table 5.1–4  Surficial Geology', 'Table 11.1–1  Wildlife Species Selected For Detailed Assessment And Selection Rationale', 'Table 12.1–1  Wildlife Species Selected For Assessment And Selection Rationale']
[]
['Table 2-1 Extent Of Land Use And Soil Units On The Project Route']
No table on page 9 is detected
['Table 5.1–5  Stations Used To Provide Climate Normal Data For The Project Area', 'Table 17.1–1  Previous Historic Resources Impact Assessments In Portions Of The Historical Resources Regional Study Area']
['Table 5.1–6  Temperature, Precipitation And Wind Speed Data For The Grand Prairie Station', 'Table 5.1–7  Temperature And Precipitation (Shining Bank) And Wind Speed (Edson) Data', 'Table 3-2 Agricultural Suitability Classes', '

['Table 18.3–1  Key Indicators And Potential Project Effects On Navigation And Navigation Safety']
No table on page 49 is detected
['Table 6.1–11  Past Spills Or Incidents Within The Lsds Of The Project Footprint, Valhalla Section', 'Table 18.4–1  Potential Effects On Navigation And Navigation Safety, Mitigation And Predicted Residual Effects']
['Table 12.3–4  Habitats Affected For The Olive-Sided Flycatcher In The Local Study Area', 'Table D-1  Water Well Drillers Records']
No table on page 51 is detected
['Table 18.5–1  Predicted Residual Effects For Navigation And Navigation Safety']
Page 52 contains an image
['Table 6.1–12  Soil Types Within The White Area Of The Elmworth Section Local Study Area']
['Table 6.1–13  Soil Map Units And Their Extent Within The White Area Of The Elmworth Section Local Study Area', 'Table 12.3–5  Habitats Affected For The Woodland Caribou In The Local Study Area']
['Table 11.3–1  Key Indicators And Potential Project Effects On Wildlife Species Of Managem

[]
No table on page 99 is detected
['Table 4.4–4  Alberta Transportation Roadwork Projects']
No table on page 100 is detected
[]
No table on page 101 is detected
['Table 6.1–30  Past Spills Or Incidents Within The Lsds Of The Project Footprint, Robb Section', 'Table 4.4–5  Number Of Applications To The Alberta Energy Regulator For Reclamation Certificates By Year', 'Table 10.5–1  Predicted Residual Effects For Wetlands']
[]
No table on page 103 is detected
['Table 6.1–31  Soil Types Within The Dismal Creek Section Local Study Area']
['Table 6.1–32  Soil Map Units And Their Extent Within The Dismal Creek Section Local Study Area', 'Table 4.4–6  Reasonably Foreseeable Projects Within The Socio-Economic Study Area']
['Table 11.6–1  Predicted Cumulative Effects For Wildlife And Wildlife Habitat', 'Table 1  Track Densities Within Grande Prairie West Elmworth Section']
No table on page 106 is detected
['Table 2  Track Densities Within Grande Prairie South Karr Section']
No table on page 107 

['Table 20.5–7  Potential Economic Annual Effects Of Spending In Alberta During The Operations Phase', 'Table 20.5–8  Construction Taxes By Type']
Page 181 contains an image
['Table 7.1–6  Ecological Land Classes Within The Karr Section Local Study Area', 'Table 20.5–9  Operations Annual Taxes By Type', 'Table 20.5–10  Ngtl Estimates Of Taxes Per Municipality']
['Table 3.2–3  Summary Of Predicted Ground-Level Concentrations At Nordegg For The Baseline Plus Project Case']
['Table 20.5–11  Potential Effects On Employment And Economy, Mitigation And Predicted Residual Effects']
[]
Page 185 contains an image
['Table 7.1–7  Ecological Land Classes Within The Deep Valley Section Local Study Area', 'Table 20.5–12  Predicted Residual Effects For Employment And Economy']
[]
[]
[]
Page 189 contains an image
[]
Page 190 contains an image
['Table 3.3–1  Summary Of Predicted Ground-Level Concentrations At Beiseker For The Project-Alone Case', 'Table 3.3–2  Summary Of Predicted Ground-Level Concentr

['Table 8-1  Amphibian Species Of Management Concern', 'Table 14.1–1  Provincial And National Greenhouse Gas Emissions']
No table on page 257 is detected
['Table 14.2–1  Emissions From Pipeline Construction', 'Table 14.2–2  Estimated Emissions For Facility Construction', 'Table 14.2–3  Emissions From Land Clearing']
No table on page 258 is detected
['Table 14.2–4  Emission Factors For The Compressors', 'Table 14.2–5  Emissions From Pipeline Inspection And Maintenance', 'Table 14.2–6  Estimated Venting And Fugitive Emissions']
No table on page 259 is detected
['Table 8.1–1  Federal And Provincial Water Quality Guidelines Used In Baseline Setting', 'Table 14.3–1  Emissions From Construction', 'Table 14.3–2  Project Greenhouse Gas Emissions From Operation Of Compressor Station Unit Additions', 'Table 14.3–3  Estimated Greenhouse Gas Emissions From Venting And Fugitive Emissions']
['Table 14.3–4  Estimated Greenhouse Gas Emissions From Project Operation', 'Table 14.3–5  Key Indicators And 

## Application for the 2017 NGTL System Expansion

In [68]:
hearing = 'Application for the 2017 NGTL System Expansion'
ngtl2017 = df[df['Application title short'] == hearing].reset_index(drop = True)
ngtl2017.head()

,page_number,final_table_title,categories,DataID_pdf,Application title short,Category
0,6,Table 8C.1 Cumulative Effects Characterization...,0.75,2748265.pdf,Application for the 2017 NGTL System Expansion,Table
1,7,Table 8C.2 Cumulative Effects Characterization...,0.75,2748265.pdf,Application for the 2017 NGTL System Expansion,Table
2,8,Table 8C.2 Cumulative Effects Characterization...,0.35,2748265.pdf,Application for the 2017 NGTL System Expansion,Table
3,9,Table 8C.3 Cumulative Effects Characterization...,0.75,2748265.pdf,Application for the 2017 NGTL System Expansion,Table
4,10,Table 8C.3 Cumulative Effects Characterization...,0.35,2748265.pdf,Application for the 2017 NGTL System Expansion,Table


### Change this folder to the path were you want the tables saved

In [ ]:
os.chdir(r'F:\Environmental Baseline Data\Version 4 - Final\CSV\ngtl2017')

### Change the dataframe name accordingly

In [6]:
file = ngtl2017['DataID_pdf'][0]
file

'3578647.pdf'

### call the main function -- pass filename and dataframe as function arguments

In [ ]:
extract_tables(file,ngtl2017)

## Application for Northwest Mainline Komie North Extension

In [21]:
hearing = 'Application for Northwest Mainline Komie North Extension'
komie_north = df[df['Application title short'] == hearing].reset_index(drop = True)
komie_north.head()

,page_number,final_table_title,categories,DataID_pdf,Application title short,Category
0,7,Table 1 Summary Of Identified Residual Project...,1.0,729032.pdf,Application for Northwest Mainline Komie North...,Table
1,9,Table 1 Summary Of Identified Residual Project...,1.0,729032.pdf,Application for Northwest Mainline Komie North...,Table
2,11,Table 1 Summary Of Identified Residual Project...,1.0,729032.pdf,Application for Northwest Mainline Komie North...,Table
3,13,Table 1 Summary Of Identified Residual Project...,1.0,729032.pdf,Application for Northwest Mainline Komie North...,Table
4,15,Table 1 Summary Of Identified Residual Project...,1.0,729032.pdf,Application for Northwest Mainline Komie North...,Table


### Change this folder to the path were you want the tables saved

In [22]:
os.chdir(r'F:\Environmental Baseline Data\Version 4 - Final\CSV\komie_north')

### Change the dataframe name accordingly

In [23]:
file = komie_north['DataID_pdf'][0]
file

'729032.pdf'

### call the main function -- pass filename and dataframe as function arguments

In [ ]:
extract_tables(file,komie_north)